## Demo

In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import pandas as pd
from config_manager import ConfigManager
from data_access import DataAccess

schema = ConfigManager.get_config('workloadConfig.schema')
table = ConfigManager.get_config('workloadConfig.table')

bikes = DataAccess().select_to_df(f"SELECT * FROM {schema}.{table}")

# bikes = pd.read_csv('./datasets/austin_bikes_final.csv')
bikes

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,614429,693,2016-03-20 19:46:36+02:00,8,5th & Bowie,8th & Congress,30.26960,-97.75332,30.26980,-97.74186
1,614430,577,2016-03-01 16:07:50+02:00,11,Barton Springs @ Kinney Ave,5th & Bowie,30.26200,-97.76118,30.26960,-97.75332
2,614431,465,2016-05-05 07:29:25+03:00,13,South Congress & Academy,Capitol Station / Congress & 11th,30.25226,-97.74854,30.27260,-97.74127
3,614432,391,2014-06-28 09:12:00+03:00,10,State Capitol Visitors Garage @ San Jacinto & ...,2nd & Congress,30.27336,-97.73805,30.26408,-97.74355
4,614433,970,2017-05-16 17:41:07+03:00,10,6th & Congress,Davis at Rainey Street,30.26822,-97.74285,30.26019,-97.73845
...,...,...,...,...,...,...,...,...,...,...
995,615424,100,2014-12-07 09:12:00+02:00,10,2nd & Congress,South Congress & James,30.26408,-97.74355,30.25103,-97.74926
996,615425,751,2017-07-19 09:02:23+03:00,3,Lavaca & 6th,4th & Congress,30.26889,-97.74525,30.26634,-97.74378
997,615426,124,2015-06-19 20:12:00+03:00,33,City Hall / Lavaca & 2nd,City Hall / Lavaca & 2nd,30.26476,-97.74678,30.26476,-97.74678
998,615427,522,2014-03-12 21:12:00+02:00,21,Pfluger Bridge @ W 2nd Street,Plaza Saltillo,30.26717,-97.75484,30.26217,-97.72743


In [64]:
from jaccard_workload_approximator import JaccardWorkloadApproximator

workload_approximator = JaccardWorkloadApproximator()
workload = workload_approximator.run(100)

============ starting iteration 1   ============
iteration took: 1.99 ms
============ starting iteration 2   ============
iteration took: 0.99 ms
============ starting iteration 3   ============
iteration took: 1.00 ms
============ starting iteration 4   ============
iteration took: 1.00 ms
============ starting iteration 5   ============
Processing query:
SELECT _id FROM public.austin_bikes_1000 WHERE start_latitude BETWEEN 30.260633808342032 AND 30.27082081826923
iteration took: 2.00 ms
============ starting iteration 6   ============
iteration took: 0.99 ms
============ starting iteration 7   ============
iteration took: 1.00 ms
============ starting iteration 8   ============
Processing query:
SELECT _id FROM public.austin_bikes_1000 WHERE end_longitude BETWEEN -97.74793219350941 AND -97.73898064555804 AND end_latitude BETWEEN 30.254714489531107 AND 30.278886171238575
iteration took: 2.02 ms
============ starting iteration 9   ============
Processing query:
SELECT _id FROM public.a

In [82]:
import re
from itertools import compress, chain

geoWorkload = []
for q in workload:
    sql = q['sql']
    geoQueries = list(compress(sql, [bool(re.search("(.*latitude*longitude*)", string)) for string in sql]))
    if any(geoQueries):
        geoWorkload.append(geoQueries)

geoWorkload = list(chain(*geoWorkload))
geoWorkload

[]

In [89]:
import re
from itertools import compress, chain

textualWorkload = []
for q in workload:
    sql = q['sql']
    textualQueries = list(compress(sql, [bool(re.search("(.*start_station_name*)", string)) for string in sql]))
    if any(textualQueries):
        textualWorkload.append(textualQueries)

textualWorkload = list(chain(*textualWorkload))
textualWorkload

["SELECT _id FROM public.austin_bikes_1000 WHERE start_station_name IN ('Barton Springs @ Kinney Ave' , 'Nueces & 3rd' , '4th & Congress' , 'Davis at Rainey Street' , 'Zilker Park' , 'Republic Square @ Guadalupe & 4th St.' , 'Capital Metro HQ - East 5th at Broadway' , 'Rainey @ River St' , 'Convention Center / 4th St. @ MetroRail' , 'Convention Center / 3rd & Trinity' , 'West & 6th St.' , 'Toomey Rd @ South Lamar' , 'South Congress & Elizabeth' , 'MoPac Pedestrian Bridge @ Veterans Drive' , '2nd & Congress' , 'San Jacinto & 8th Street' , 'Trinity & 6th Street' , 'Pease Park' , 'Barton Springs & Riverside' , 'State Parking Garage @ Brazos & 18th' , '6th & Navasota St.' , '17th & Guadalupe' , 'Plaza Saltillo' , 'Pfluger Bridge @ W 2nd Street' , 'Nueces @ 3rd' , 'Republic Square @ Federal Courthouse Plaza' , 'Medina & East 6th' , 'South Congress & James' , 'East 6th & Pedernales St.' , 'Republic Square' , 'Capitol Station / Congress & 11th' , 'UT West Mall @ Guadalupe' , 'East 11th St. & 

In [60]:
from saqp_manager import SaqpManager

k = 20
saqp_manager = SaqpManager([query['result'] for query in workload], [query['frequency'] for query in workload], k)
sample = saqp_manager.get_sample()

Start running LazyGreedy with type UC
============ starting iteration 1   ============
Updated delta[0]
Updated delta[1]
Updated delta[2]
Updated delta[3]
Updated delta[4]
Updated delta[5]
Updated delta[6]
Updated delta[7]
Updated delta[8]
Updated delta[9]
Updated delta[10]
Updated delta[11]
Updated delta[12]
Updated delta[13]
Updated delta[14]
Updated delta[15]
Updated delta[16]
Updated delta[17]
Updated delta[18]
Updated delta[19]
Updated delta[20]
Updated delta[21]
Updated delta[22]
Updated delta[23]
Updated delta[24]
Updated delta[25]
Updated delta[26]
Updated delta[27]
Updated delta[28]
Updated delta[29]
Updated delta[30]
Updated delta[31]
Updated delta[32]
Updated delta[33]
Updated delta[34]
Updated delta[35]
Updated delta[36]
Updated delta[37]
Updated delta[38]
Updated delta[39]
Updated delta[40]
Updated delta[41]
Updated delta[42]
Updated delta[43]
Updated delta[44]
Updated delta[45]
Updated delta[46]
Updated delta[47]
Updated delta[48]
Updated delta[49]
Updated delta[50]
Updat

In [76]:
df = pd.DataFrame(sample)
df

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,615458,124,2016-09-11 10:24:45+03:00,10,Convention Center / 4th St. @ MetroRail,City Hall / Lavaca & 2nd,30.26483,-97.73900,30.26476,-97.74678
1,615479,414,2016-05-18 18:31:43+03:00,3,Pfluger Bridge @ W 2nd Street,3rd & West,30.26717,-97.75484,30.26780,-97.75189
2,615464,352,2014-07-18 08:12:00+03:00,29,Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,30.27260,-97.74127,30.27260,-97.74127
3,615523,966,2015-03-18 20:12:00+02:00,4,4th & Congress,Convention Center / 4th St. @ MetroRail,30.26634,-97.74378,30.26483,-97.73900
4,615461,646,2015-12-02 17:12:27+02:00,7,5th & Bowie,Palmer Auditorium,30.26960,-97.75332,30.25971,-97.75346
5,615432,445,2016-03-28 14:41:52+03:00,92,Rainey St @ Cummings,Rainey St @ Cummings,30.25579,-97.73982,30.25579,-97.73982
6,615448,893,2016-09-30 15:38:03+03:00,36,2nd & Congress,2nd & Congress,30.26408,-97.74355,30.26408,-97.74355
7,615446,373,2016-06-12 18:38:54+03:00,13,Convention Center / 4th St. @ MetroRail,Davis at Rainey Street,30.26483,-97.73900,30.26019,-97.73845
8,615442,263,2017-01-29 11:04:39+02:00,54,South Congress & Academy,South Congress & Barton Springs at the Austin ...,30.25226,-97.74854,30.25839,-97.74592
9,615528,855,2017-03-19 15:24:58+02:00,40,Barton Springs & Riverside,Long Center @ South 1st & Riverside,30.25870,-97.74872,30.25941,-97.74971


In [77]:
random_sample = DataAccess().select(f"SELECT * FROM {schema}.{table} ORDER BY RANDOM() LIMIT {saqp_manager.k}")
df2 = pd.DataFrame(random_sample)

In [78]:
score_func = saqp_manager.saqpParAdapter.get_gain_function()
print(f"Sample score for our sample: {score_func(sample)}")
print(f"Sample score for random sample: {score_func(random_sample)}")

Sample score for our sample: 42620.95909247393358081018815
Sample score for random sample: 41161.04880396528043042053545


In [68]:
query = input('enter query please...').replace("_id", "*")
query

'SELECT * FROM public.austin_bikes_100 WHERE bikeid BETWEEN 364.2393452967148 AND 1408.728611822106'

In [79]:
from data_access import DataAccess

dataAccess = DataAccess()
truth_query_df = dataAccess.select_to_df(query)
truth_query_df

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,615432,445,2016-03-28 14:41:52+03:00,92,Rainey St @ Cummings,Rainey St @ Cummings,30.25579,-97.73982,30.25579,-97.73982
1,615433,894,2015-03-07 16:12:00+02:00,44,Rainey St @ Cummings,Rainey St @ Cummings,30.25579,-97.73982,30.25579,-97.73982
2,615435,832,2016-09-27 20:45:17+03:00,4,4th & Congress,Republic Square @ Federal Courthouse Plaza,30.26634,-97.74378,30.26753,-97.74805
3,615436,772,2015-08-25 18:12:00+03:00,4,East 4th & Chicon,East 6th & Pedernales St.,30.25987,-97.72373,30.25895,-97.71475
4,615438,428,2014-08-22 14:12:00+03:00,9,ACC - Rio Grande & 12th,Convention Center / 4th St. @ MetroRail,30.27595,-97.74739,30.26483,-97.73900
5,615440,371,2016-08-09 17:53:52+03:00,13,Convention Center / 4th St. @ MetroRail,Barton Springs @ Kinney Ave,30.26483,-97.73900,30.26200,-97.76118
6,615441,434,2017-03-19 02:21:45+02:00,63,Rainey St @ Cummings,Medina & East 6th,30.25579,-97.73982,30.26455,-97.73165
7,615443,860,2015-04-09 18:12:00+03:00,5,5th & Bowie,West & 6th St.,30.26960,-97.75332,30.27041,-97.75046
8,615446,373,2016-06-12 18:38:54+03:00,13,Convention Center / 4th St. @ MetroRail,Davis at Rainey Street,30.26483,-97.73900,30.26019,-97.73845
9,615448,893,2016-09-30 15:38:03+03:00,36,2nd & Congress,2nd & Congress,30.26408,-97.74355,30.26408,-97.74355


In [80]:
import pandasql as ps

sample_query_df = ps.sqldf(query.replace(f"{schema}.{table}", "df"))
sample_query_df

,_id,bikeid,start_time,duration_minutes,start_station_name,end_station_name,start_latitude,start_longitude,end_latitude,end_longitude
0,615479,414,2016-05-18 18:31:43.000000,3,Pfluger Bridge @ W 2nd Street,3rd & West,30.26717,-97.75484,30.26780,-97.75189
1,615523,966,2015-03-18 20:12:00.000000,4,4th & Congress,Convention Center / 4th St. @ MetroRail,30.26634,-97.74378,30.26483,-97.73900
2,615461,646,2015-12-02 17:12:27.000000,7,5th & Bowie,Palmer Auditorium,30.26960,-97.75332,30.25971,-97.75346
3,615432,445,2016-03-28 14:41:52.000000,92,Rainey St @ Cummings,Rainey St @ Cummings,30.25579,-97.73982,30.25579,-97.73982
4,615448,893,2016-09-30 15:38:03.000000,36,2nd & Congress,2nd & Congress,30.26408,-97.74355,30.26408,-97.74355
5,615446,373,2016-06-12 18:38:54.000000,13,Convention Center / 4th St. @ MetroRail,Davis at Rainey Street,30.26483,-97.73900,30.26019,-97.73845
6,615528,855,2017-03-19 15:24:58.000000,40,Barton Springs & Riverside,Long Center @ South 1st & Riverside,30.25870,-97.74872,30.25941,-97.74971
7,615485,712,2015-07-31 14:12:00.000000,5,4th & Congress,5th & Bowie,30.26634,-97.74378,30.26960,-97.75332
8,615466,555,2016-10-30 16:26:59.000000,21,Palmer Auditorium,Palmer Auditorium,30.25971,-97.75346,30.25971,-97.75346
9,615457,511,2016-02-07 14:32:38.000000,15,Plaza Saltillo,Plaza Saltillo,30.26217,-97.72743,30.26217,-97.72743


In [ ]:
# TODO here we want data from the histogram to complete the answer of the query over the sample

In [81]:
import pandasql as ps

random_query_df = ps.sqldf(query.replace(f"{schema}.{table}", "df2"))

In [82]:
sample_query_score = saqp_manager.saqpParAdapter.query_result_score(sample_query_df.to_dict('records'),
                                                                    truth_query_df.to_dict('records'))
random_query_score = saqp_manager.saqpParAdapter.query_result_score(random_query_df.to_dict('records'),
                                                                    truth_query_df.to_dict('records'))
print(f"Query result score for our sample: {sample_query_score}")
print(f"Query result score for random sample: {random_query_score}")

Query result score for our sample: 0.20390607576385128
Query result score for random sample: 0.16331602255710964


In [83]:
len(sample) / dataAccess.select_one(f"SELECT count(*) as count from public.{table}")['count']

0.2

In [84]:
len(sample_query_df.index) / len(truth_query_df.index)

0.2

In [86]:
len(random_query_df.index) / len(truth_query_df.index)

0.16363636363636364

In [ ]:
# TODO compare with other baselines like k-means